In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import jax
#---------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#---------------------------------------
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#---------------------------------------
import warnings
warnings.filterwarnings("ignore")

2025-12-27 01:03:08.683106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766797389.269188      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766797389.408554      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
temp_dir = "/kaggle/input/templates/Templates"
dataset_dir = "/kaggle/input/cytomorphology/PKG - AML-Cytomorphology_MLL_Helmholtz_v1/data"
excel_path = "/kaggle/input/preprocessed-xlsx/Preprocessed clinical data.xlsx"

image11v = 0,0,0
image12v = 10,10,10
image21v = 20,20,20
image22v = 30,30,30
sexv = 50,50,50
agev = 70,70,70
per_ulv = 90,90,90
pb_myebv = 100,100,100
pb_promv = 120,120,120
pb_myelv = 140,140,140
pb_metav = 160,160,160
pb_neutbv = 180,180,180
pb_neutsv = 200,200,200
pb_eosiv = 220,220,220
pb_basov = 240,240,240
pb_monov = 250,250,250
pb_lymtv = 230,230,230
pb_lymarv = 210,210,210
pb_lymanv = 190,190,190
pb_ov = 170,170,170

In [3]:
datamap = [
    ["image11", image11v],
    ["image12", image12v],
    ["image21", image21v],
    ["image22", image22v],
    ["sex", sexv],
    ["age", agev],
    ["per_ul", per_ulv],
    ["pb_myeb", pb_myebv],
    ["pb_prom", pb_promv],
    ["pb_myel", pb_myelv],
    ["pb_meta", pb_metav],
    ["pb_neutb", pb_neutbv],
    ["pb_neuts", pb_neutsv],
    ["pb_eosi", pb_eosiv],
    ["pb_baso", pb_basov],
    ["pb_mono", pb_monov],
    ["pb_lymt", pb_lymtv],
    ["pb_lymar", pb_lymarv],
    ["pb_lyman", pb_lymanv],
    ["pb_o", pb_ov]
]

imagemap = [
    ["pb_myeb", (255,0,0)],
    ["pb_prom", (255,128,0)],
    ["pb_myel", (255,255,0)],
    ["pb_meta", (128,255,0)],
    ["pb_neutb", (0,255,0)],
    ["pb_neuts", (0,255,128)],
    ["pb_eosi", (0,255,255)],
    ["pb_baso", (0,128,255)],
    ["pb_mono", (0,0,255)],
    ["pb_lymt", (128,0,255)],
    ["pb_lymar", (255,0,255)],
    ["pb_lyman", (255,0,128)],
    ["pb_o", (255,255,255)]
]

In [4]:
### function to return mapping

def check(x, arr=datamap):
    mapping = dict(arr)
    
    # Add reverse mappings too
    reverse_mapping = {v: k for k, v in arr}
    mapping.update(reverse_mapping)

    if x in mapping:
        return mapping[x]
    else:
        raise KeyError(f"No mapping found for '{x}'")

### function to replace a regions with a certain color

def readTemplate(template_path):
    image = cv2.imread(template_path)
    h, w, _ = image.shape
    
    splitData = np.zeros((len(datamap), h, w), dtype=np.uint8)
    
    for i in range(h):
        for j in range(w):
            pixel = tuple(image[i, j])          # Make hashable
            dataType = check(pixel)
            index = next(k for k, pair in enumerate(datamap) if pair[0] == dataType)
            splitData[index, i, j] = 1

    return splitData

def altData(data, templateData, arr1 = datamap, arr2 = imagemap):
    returningData = np.zeros((16,164,164,3))

    #print(data.shape)

    #Handling sex
    if data[0] == 0:
        returningData[0] = (0,128,255)
    else:
        returningData[0] = (255,0,128)

    #age
    returningData[1] = (data[1] / 100)*np.array([111, 0, 255])
    
    #per_ul
    returningData[2] = (data[2]/270)*np.array([128,255,0])

    #rest of the data follow the same pattern
    for i in range(len(imagemap)):
        #print(i)
        imageMapVAR = np.array([imagemap[i][1]])
        Template = np.array([templateData[i+4]])

        imageMapVAR = imageMapVAR.reshape(1,1,3)
        Template = Template[...,None]

        #print((data[i+1]/100))
        plt.imshow(imageMapVAR)
        plt.imshow(Template[0])
        #print((data[i+1]/100).shape)
        #print(imageMapVAR.shape)
        #print(Template.shape)
        
        returningData[i] = (data[i+1]/100)*imageMapVAR*Template
    
    return returningData

In [5]:
"""data_class_list = os.listdir(dataset_dir)
templates = os.listdir(temp_dir)

#print(templates)

newDataPath = "/kaggle/working/NewDataset/"

df = pd.read_excel(excel_path, engine='openpyxl')

for template in templates:
    template_path = os.path.join(temp_dir,template)

    newpath = os.path.join(newDataPath,template)
    #print(template_path)
    #print(newpath)

    for index, row in df.iterrows():
        ID = row['patient_id']
        sex = row['sex_1f_2m']
        age = row['age']
        data = row[4:18]
        data = data.fillna(0)
        fullData = sex + age + data
        #print(fullData)
        for classs in data_class_list:
            class_path = os.path.join(dataset_dir,classs)

            newClassPath = os.path.join(newpath,classs)
            
            patient_path = os.path.join(class_path,ID)
            
            if os.path.isdir(patient_path):
                newPatientPath = os.path.join(newClassPath,ID)
                os.makedirs(newPatientPath, exist_ok=True)
                image_list = os.listdir(patient_path)

                for image in image_list:
                    image_path = os.path.join(patient_path,image)
                    filename = os.path.splitext(image)[0] + ".png"
                    newImagePath = os.path.join(newPatientPath, filename)

                    #print(image_path)

                    image = cv2.imread(image_path)
                    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                    #print(image.shape)
        
                    part_11 = image[0:72, 0:72]
                    part_12 = image[0:72, 72:144]
                    part_21 = image[72:144, 0:72]
                    part_22 = image[72:144, 72:144]


                    parts = [part_11,part_12,part_21,part_22]

                    image_values = [image11v,image12v,image21v,image22v]        
                    
                    templateData = readTemplate(template_path)
                    #print(templateData.shape)
                    
                    imageArray = np.zeros((4, templateData[0].shape[0], templateData[0].shape[1],3))
                    

                    for sector in image_values:
                        rows, cols = np.where(templateData[image_values.index(sector)] == 1)
                        r_min, r_max = rows.min(), rows.max() + 1
                        c_min, c_max = cols.min(), cols.max() + 1

                        imageArray[image_values.index(sector)][r_min:r_max, c_min:c_max] = parts[image_values.index(sector)]
                    dataPoints = altData(fullData,templateData)

                    #print(imageArray.shape)

                    Final_image3D = np.concatenate((imageArray, dataPoints), axis=0)

                    Final_image = np.sum(Final_image3D, axis=0)
                    os.makedirs(newPatientPath, exist_ok=True)
                    cv2.imwrite(newImagePath, Final_image)
                    #print("==============================================================================")"""

'data_class_list = os.listdir(dataset_dir)\ntemplates = os.listdir(temp_dir)\n\n#print(templates)\n\nnewDataPath = "/kaggle/working/NewDataset/"\n\ndf = pd.read_excel(excel_path, engine=\'openpyxl\')\n\nfor template in templates:\n    template_path = os.path.join(temp_dir,template)\n\n    newpath = os.path.join(newDataPath,template)\n    #print(template_path)\n    #print(newpath)\n\n    for index, row in df.iterrows():\n        ID = row[\'patient_id\']\n        sex = row[\'sex_1f_2m\']\n        age = row[\'age\']\n        data = row[4:18]\n        data = data.fillna(0)\n        fullData = sex + age + data\n        #print(fullData)\n        for classs in data_class_list:\n            class_path = os.path.join(dataset_dir,classs)\n\n            newClassPath = os.path.join(newpath,classs)\n            \n            patient_path = os.path.join(class_path,ID)\n            \n            if os.path.isdir(patient_path):\n                newPatientPath = os.path.join(newClassPath,ID)\n         

In [ ]:
# Complete GPU-accelerated dataset generation script for Kaggle
# Assumptions:
#  - CUDA-enabled GPU is available in the runtime
#  - OpenCV (cv2), pandas, numpy are available (standard Kaggle)
#  - The user-provided variables (datamap, imagemap, image11v, etc.) are present
#  - Template images are same resolution as code expects (h x w; code reads them dynamically)

import os
import sys
import math
import subprocess
from tqdm import tqdm

# Try import cupy; if not installed try to pip install a GPU build.
try:
    import cupy as cp
except Exception as e:
    print("CuPy not found. Attempting to install cupy-cuda from pip (this may take a while)...")
    # choose the appropriate cupy-cuda version for the Kaggle CUDA version; common is cuda11x or cuda12x.
    # If you know exact cuda version, change `cupy-cuda12x` accordingly.
    install_cmd = [sys.executable, "-m", "pip", "install", "--no-cache-dir", "cupy-cuda12x"]
    subprocess.check_call(install_cmd)
    import cupy as cp

import numpy as np
import cv2
import pandas as pd

# ---------------------------
# User-provided variables
# (You already gave these values; they are repeated here for the script to be self-contained)
# ---------------------------
temp_dir = "/kaggle/input/templates/Templates"
dataset_dir = "/kaggle/input/cytomorphology/PKG - AML-Cytomorphology_MLL_Helmholtz_v1/data"
excel_path = "/kaggle/input/preprocessed-xlsx/Preprocessed clinical data.xlsx"

image11v = (0,0,0)
image12v = (10,10,10)
image21v = (20,20,20)
image22v = (30,30,30)
sexv = (50,50,50)
agev = (70,70,70)
per_ulv = (90,90,90)
pb_myebv = (100,100,100)
pb_promv = (120,120,120)
pb_myelv = (140,140,140)
pb_metav = (160,160,160)
pb_neutbv = (180,180,180)
pb_neutsv = (200,200,200)
pb_eosiv = (220,220,220)
pb_basov = (240,240,240)
pb_monov = (250,250,250)
pb_lymtv = (230,230,230)
pb_lymarv = (210,210,210)
pb_lymanv = (190,190,190)
pb_ov = (170,170,170)

datamap = [
    ["image11", image11v],
    ["image12", image12v],
    ["image21", image21v],
    ["image22", image22v],
    ["sex", sexv],
    ["age", agev],
    ["per_ul", per_ulv],
    ["pb_myeb", pb_myebv],
    ["pb_prom", pb_promv],
    ["pb_myel", pb_myelv],
    ["pb_meta", pb_metav],
    ["pb_neutb", pb_neutbv],
    ["pb_neuts", pb_neutsv],
    ["pb_eosi", pb_eosiv],
    ["pb_baso", pb_basov],
    ["pb_mono", pb_monov],
    ["pb_lymt", pb_lymtv],
    ["pb_lymar", pb_lymarv],
    ["pb_lyman", pb_lymanv],
    ["pb_o", pb_ov]
]

imagemap = [
    ["pb_myeb", (255,0,0)],
    ["pb_prom", (255,128,0)],
    ["pb_myel", (255,255,0)],
    ["pb_meta", (128,255,0)],
    ["pb_neutb", (0,255,0)],
    ["pb_neuts", (0,255,128)],
    ["pb_eosi", (0,255,255)],
    ["pb_baso", (0,128,255)],
    ["pb_mono", (0,0,255)],
    ["pb_lymt", (128,0,255)],
    ["pb_lymar", (255,0,255)],
    ["pb_lyman", (255,0,128)],
    ["pb_o", (255,255,255)]
]

# Which labels represent the four image quadrants (BGR tuples)
image_values = [image11v, image12v, image21v, image22v]

# Output path
newDataPath = "/kaggle/working/NewDataset_gpu/"
os.makedirs(newDataPath, exist_ok=True)

# ---------------------------
# Helper functions (GPU)
# ---------------------------

# Create quick lookup maps (label -> color tuple) on CPU (small structures)
label_to_color = {label: tuple(color) for label, color in datamap}
label_to_index = {label: idx for idx, (label, _) in enumerate(datamap)}

# NOTE: cv2.imread returns BGR. The datamap tuples are interpreted as BGR tuples here.
# If your datamap colors are RGB instead, swap channels when comparing (or use cv2.cvtColor).

def readTemplate_gpu(template_path):
    """
    Reads template using OpenCV then creates a boolean mask per datamap key on the GPU.
    Returns: templateData -> cp.ndarray shape (len(datamap), h, w), dtype=uint8 (0/1)
    """
    img = cv2.imread(template_path)  # BGR uint8
    if img is None:
        raise FileNotFoundError(f"Template not found: {template_path}")

    h, w, _ = img.shape

    # Move image to GPU (uint8)
    img_gpu = cp.asarray(img, dtype=cp.uint8)  # shape (h,w,3)

    n_keys = len(datamap)
    templateData = cp.zeros((n_keys, h, w), dtype=cp.uint8)

    # Vectorized per-key masks
    for idx, (label, color_tuple) in enumerate(datamap):
        # color_tuple assumed to be (B, G, R) matching cv2.imread order
        b, g, r = color_tuple
        # build boolean mask where all channels match
        mask = (img_gpu[:,:,0] == int(b)) & (img_gpu[:,:,1] == int(g)) & (img_gpu[:,:,2] == int(r))
        templateData[idx] = mask.astype(cp.uint8)

    return templateData  # on GPU


def altData_gpu(fullData_list, templateData):
    """
    Build the data overlays (the colored channel contributions) on GPU.
    fullData_list: CPU list or 1D numpy array of numeric values [sex, age, ... other features ...]
    templateData: cp.ndarray (n_keys, h, w) - provided by readTemplate_gpu
    Returns cp.ndarray shape (4 + len(imagemap), h, w, 3), dtype=float32
    Index mapping:
      0..3 => reserved for quadrants (we will return only the "data overlays" portion here starting index 4)
    """
    # Convert fullData to GPU array (float)
    fd = cp.asarray(fullData_list, dtype=cp.float32)

    h = int(templateData.shape[1])
    w = int(templateData.shape[2])

    n_imagemap = len(imagemap)
    out_count = 4 + n_imagemap
    returningData = cp.zeros((out_count, h, w, 3), dtype=cp.float32)

    # 0: sex coloring (using provided sexv colors as BGR tuples)
    # We expect fullData_list[0] to be sex (0 or 1 or whatever numeric). Normalize / scale as per previous logic.
    sex_val = float(fullData_list[0]) if len(fullData_list) > 0 else 0.0
    # If sex_val == 0 -> use one color else other (match prior logic)
    sex_color = cp.array([0,128,255], dtype=cp.float32) if sex_val == 0 else cp.array([255,0,128], dtype=cp.float32)
    returningData[0] = sex_color.reshape(1,1,3)

    # 1: age (previous code used (age/100)*[111,0,255])
    age_val = float(fullData_list[1]) if len(fullData_list) > 1 else 0.0
    returningData[1] = (age_val / 100.0) * cp.array([111.0, 0.0, 255.0], dtype=cp.float32).reshape(1,1,3)

    # 2: per_ul (used denom 270 in earlier code)
    per_ul_val = float(fullData_list[2]) if len(fullData_list) > 2 else 0.0
    returningData[2] = (per_ul_val / 270.0) * cp.array([128.0,255.0,0.0], dtype=cp.float32).reshape(1,1,3)

    # Index 3 left unused for symmetry with "4 quadrants" in main assembly (we can leave zeros)
    returningData[3] = cp.zeros((1,1,3), dtype=cp.float32)

    # Now for each imagemap entry, scale the color by corresponding fullData value, and mask with templateData
    # We need to map imagemap entry label to a position in fullData_list.
    # In your original code you used data[i+1] indexing; here we assume the sequence in fullData_list follows:
    # [sex, age, per_ul, pb_myeb, pb_prom, pb_myel, pb_meta, pb_neutb, pb_neuts, pb_eosi, pb_baso,
    #  pb_mono, pb_lymt, pb_lymar, pb_lyman, pb_o]
    # That is: fullData_list[3 + j] corresponds to imagemap[j]
    for j, (label, color_tuple) in enumerate(imagemap):
        # get the corresponding numeric feature value (safe-check index)
        feature_index = 3 + j  # as per mapping assumption above
        feature_val = float(fullData_list[feature_index]) if feature_index < len(fullData_list) else 0.0
        # color as float on GPU (BGR)
        color_vec = cp.asarray(color_tuple, dtype=cp.float32).reshape(1,1,3)
        # mask uses templateData entry for the corresponding datamap label index:
        # We need to find which index in datamap corresponds to this label
        if label not in label_to_index:
            # skip if absent
            continue
        datamap_idx = label_to_index[label]
        mask = templateData[datamap_idx].astype(cp.float32)  # shape (h,w)
        mask3 = mask[..., None]  # shape (h,w,1)

        # scaled contribution
        contribution = (feature_val / 100.0) * color_vec * mask3  # (h,w,3)
        returningData[4 + j] = contribution

    return returningData  # shape (4 + n_imagemap, h, w, 3)


# ---------------------------
# Main processing loop
# ---------------------------

# Read dataframe
df = pd.read_excel(excel_path, engine='openpyxl')

# Templates list
templates = [f for f in os.listdir(temp_dir) if os.path.isfile(os.path.join(temp_dir, f))]

data_class_list = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

# For faster repeated access, build mapping from datamap tuples -> datamap index
# datamap is list of [label, (B,G,R)]
color_to_datamap_index = {tuple(pair[1]): idx for idx, pair in enumerate(datamap)}

# Iterate templates
for template_name in tqdm(templates, desc="Templates"):
    template_path = os.path.join(temp_dir, template_name)
    print(f"Processing template: {template_path}")

    # Prepare output folder per-template
    newpath = os.path.join(newDataPath, template_name)
    os.makedirs(newpath, exist_ok=True)

    # Read template once (GPU)
    templateData = readTemplate_gpu(template_path)  # shape (len(datamap), h, w) on GPU
    h = int(templateData.shape[1])
    w = int(templateData.shape[2])

    # precompute indices of quadrant datamap entries (so we know which template masks correspond to quadrant)
    quadrant_labels = ['image11', 'image12', 'image21', 'image22']
    quadrant_datamap_indices = [label_to_index.get(lbl) for lbl in quadrant_labels]
    # If any is None, script will still attempt to run but will skip empty ones.

    # iterate over patient rows
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Patients", leave=False):
        ID = str(row['patient_id'])
        sex = row['sex_1f_2m']
        age = row['age']
        data_series = row[4:18].fillna(0)  # as user did earlier

        # Build fullData_list: [sex, age] + other fields in same order expected
        # We need this order to align with altData_gpu's expectations (feature indices).
        # The user previously used: fullData = sex + age + data  (ambiguous), so we will construct:
        fullData_list = [float(sex), float(age)] + [float(x) for x in data_series.tolist()]

        # For each data class folder (classs)
        for classs in data_class_list:
            class_path = os.path.join(dataset_dir, classs)
            newClassPath = os.path.join(newpath, classs)
            patient_path = os.path.join(class_path, ID)

            if not os.path.isdir(patient_path):
                continue

            newPatientPath = os.path.join(newClassPath, ID)
            os.makedirs(newPatientPath, exist_ok=True)

            image_list = [f for f in os.listdir(patient_path) if os.path.isfile(os.path.join(patient_path, f))]

            for img_name in image_list:
                image_path = os.path.join(patient_path, img_name)
                filename = os.path.splitext(img_name)[0] + ".png"
                newImagePath = os.path.join(newPatientPath, filename)

                # read whole image (BGR)
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Warning: couldn't read {image_path}. Skipping.")
                    continue

                # If the images are 144x144 and split into 4 quadrants of 72x72 each, as in the original code:
                # Extract quadrants (careful with indexing)
                ih, iw = image.shape[:2]
                # compute quadrant sizes dynamically (assume 2x2 tiling)
                ch = ih // 2
                cw = iw // 2
                part_11 = image[0:ch, 0:cw]           # top-left
                part_12 = image[0:ch, cw:2*cw]        # top-right
                part_21 = image[ch:2*ch, 0:cw]        # bottom-left
                part_22 = image[ch:2*ch, cw:2*cw]     # bottom-right

                parts = [part_11, part_12, part_21, part_22]  # CPU numpy arrays

                # Prepare GPU imageArray for quadrants: shape (4, h, w, 3)
                imageArray = cp.zeros((4, h, w, 3), dtype=cp.float32)

                # For each quadrant, find region in templateData and paste the part (moved to GPU)
                for q_idx, datamap_idx in enumerate(quadrant_datamap_indices):
                    if datamap_idx is None:
                        continue
                    mask = templateData[datamap_idx]  # GPU (h, w) with 0/1

                    # If mask is all zeros, skip
                    if int(cp.sum(mask)) == 0:
                        continue

                    # compute bounding box on mask
                    rows, cols = cp.where(mask == 1)
                    r_min = int(rows.min())
                    r_max = int(rows.max()) + 1
                    c_min = int(cols.min())
                    c_max = int(cols.max()) + 1

                    # get CPU part and move to GPU
                    part_cpu = parts[q_idx]
                    # If part shape mismatches bounding box, we will try to resize to fit
                    ph, pw = part_cpu.shape[:2]
                    box_h = r_max - r_min
                    box_w = c_max - c_min
                    if (ph != box_h) or (pw != box_w):
                        # resize part to fit bounding box
                        part_resized = cv2.resize(part_cpu, (box_w, box_h), interpolation=cv2.INTER_LINEAR)
                    else:
                        part_resized = part_cpu

                    # move to GPU and convert to float32
                    part_gpu = cp.asarray(part_resized.astype(np.float32))

                    # paste into the imageArray[ q_idx, r_min:r_max, c_min:c_max ]
                    imageArray[q_idx, r_min:r_max, c_min:c_max, :] = part_gpu

                # Build data overlays (GPU)
                dataPoints = altData_gpu(fullData_list, templateData)  # shape (4 + n_imagemap, h, w, 3)

                # Now concatenate imageArray (4, ...) with the dataPoints (4 + n, ...)
                # To avoid duplicating the first 4 from dataPoints (we kept placeholders), we only take indexes 4: of dataPoints
                overlay_part = dataPoints[4:]  # shape (n_imagemap, h, w, 3)
                Final_image3D = cp.concatenate((imageArray, overlay_part), axis=0)  # axis=0 => channels

                # Sum over channel dimension to compose final image
                Final_image = cp.sum(Final_image3D, axis=0)  # shape (h, w, 3), float32

                # Clip and convert to uint8
                Final_image_cpu = cp.asnumpy(Final_image)
                Final_image_cpu = np.clip(Final_image_cpu, 0, 255).astype(np.uint8)

                # Save
                cv2.imwrite(newImagePath, Final_image_cpu)

    print(f"Finished template: {template_name}")

print("All templates processed. Output at:", newDataPath)


Templates:   0%|          | 0/5 [00:00<?, ?it/s]

Processing template: /kaggle/input/templates/Templates/Template_5.png



Patients:  71%|███████▏  | 135/189 [20:31<08:54,  9.89s/it]

In [ ]:
!zip -rq /kaggle/working/Dataset_5_Templates.zip /kaggle/working/NewDataset_gpu

In [ ]:
print"DONE"